In [ ]:
#uncomment this line if you get import errors
!pip install tensorflowjs

In [ ]:
import glob
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
from keras import models, layers
import tensorflowjs as tfjs
import zipfile
from zipfile import ZipFile

The following three code cells involve loading the images, converting them to the Lab and greyscale colorspaces, and dividing them into testing and training feature/label sets.

In [ ]:
def getColorBWImages(filepath):
  rgb_img = cv2.imread(filepath) #take in the BGR image in the form of a numpy array
  gray_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2GRAY) #convert to grayscale using cv2
  lab_img = cv2.cvtColor(rgb_img, cv2.COLOR_BGR2LAB)
  #rgb_img = cv2.resize(rgb_img,(224,224))
  #gray_img = cv2.resize(gray_img,(224,224))
  gray_img = gray_img.reshape(224, 224, 1) / 255.; #TODO: check
  lab_img = lab_img[:, :, 1:3] / 255.
  return gray_img, lab_img #gray image has shape (x, y), ab image has shape (x, y, 2)

#combine L and ab channels into a single image
def combineLab(L, ab):
  outLab = np.empty((224, 224, 3))
  outLab[:, :, 1:3] = ab
  outLab[:, :, 0] = L.reshape(224, 224)
  #outLab[:, :, 0] = np.full((224, 224), .5)
  outLab = (outLab * 255).astype(np.uint8);
  return cv2.cvtColor(outLab, cv2.COLOR_LAB2RGB)

#resize image
def resize(filepath):
  in_path = glob.glob(os.path.join(filepath,'*.jpg'))
  for i in range(0, len(in_path)):
    rgb_img = cv2.imread(in_path[i]) #take in the BGR image in the form of a numpy array
    rgb_img = cv2.resize(rgb_img,(224,224))
    cv2.imwrite(in_path[i], rgb_img)

#resize("drive/MyDrive/ML/landscapes_some")

In [ ]:
with ZipFile('landscapes2000.zip', 'r') as zipObj:
  zipObj.extractall('')

Training data uses a custom data generator to load data from the disk during training. Testing data isn't that large, so the old system is used and data is loaded into RAM all at once.

In [ ]:
features = dict()
features['training'] = list()
features['testing'] = list()

training_path = glob.glob(os.path.join("training",'*.jpg'))
testing_path = glob.glob(os.path.join("testing",'*.jpg'))

for path in training_path:
    features['training'].append(path);
for path in testing_path:
    features['testing'].append(path);

In [ ]:
#Data generator template provided by https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size=64, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, 224, 224, 1))
        y = np.empty((self.batch_size, 224, 224, 2))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            gray_img, lab_img = getColorBWImages(ID);
            X[i] = gray_img
            y[i] = lab_img

        return X, y

The following cells set up the model, train it, evaluate it using testing data, and show a sample with one image in the testing data.

In [ ]:
#create the model used for colorizing images
input_layer = layers.Input(shape=(224,224,1))
encoder_layer = layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(input_layer)
encoder_layer = layers.MaxPooling2D((2, 2), padding='same')(encoder_layer)
encoder_layer = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(encoder_layer)
encoder_layer = layers.MaxPooling2D((2, 2), padding='same')(encoder_layer)
encoder_layer = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(encoder_layer)
encoder_layer = layers.MaxPooling2D((2, 2), padding='same')(encoder_layer)
encoder_layer = layers.Conv2D(256, (3, 3), activation = 'relu', padding = 'same')(encoder_layer)
encoder_layer = layers.MaxPooling2D((2, 2), padding='same')(encoder_layer)

decoder_layer = layers.UpSampling2D((2, 2))(encoder_layer)
decoder_layer = layers.Conv2D(128, (3, 3), activation = 'relu', padding = 'same')(decoder_layer)
decoder_layer = layers.UpSampling2D((2, 2))(decoder_layer)
decoder_layer = layers.Conv2D(64, (3, 3), activation = 'relu', padding = 'same')(decoder_layer)
decoder_layer = layers.UpSampling2D((2, 2))(decoder_layer)
decoder_layer = layers.Conv2D(32, (3, 3), activation = 'relu', padding = 'same')(decoder_layer)
decoder_layer = layers.UpSampling2D((2, 2))(decoder_layer)
decoder_layer = layers.Conv2D(16, (3, 3), activation = 'relu', padding = 'same')(decoder_layer)
output_layer = layers.Conv2D(2, (3, 3), padding = 'same', activation = 'sigmoid')(decoder_layer)

model = models.Model(input_layer, output_layer)
model.compile(optimizer = 'rmsprop', loss = 'mean_squared_error')
#model.summary()

In [ ]:
training_generator = DataGenerator(features['training'])
validation_generator = DataGenerator(features['testing'])

model.fit(training_generator, epochs=18)
model.save('landscapes.keras')
tfjs.converters.save_keras_model(model, 'landscapes.json') #btw, I added this line for exporting the model to js. Comment this out if you're loading an existing model

In [ ]:
#evaluate the model with both the training and testing data
model.evaluate(training_generator)
model.evaluate(validation_generator)

In [ ]:
#test out the model with a specific image (at index n)
n = 16
gray_img, lab_img = getColorBWImages(features['testing'][n])

out = model.predict(gray_img.reshape(1, 224, 224, 1))

plt.figure(figsize=(12, 4), dpi=100)
plt.subplot(1, 3, 1)
plt.imshow(gray_img.reshape(224, 224), cmap='gray')
plt.subplot(1, 3, 2)

plt.imshow(combineLab(gray_img, out))
plt.subplot(1, 3, 3)
plt.imshow(combineLab(gray_img, lab_img))